In [1]:
import polars as pl
import mnemosyne as ms
import lz4.frame
import boto3
from datetime import date as Date, datetime as Datetime
from pprint import pprint
from tqdm.autonotebook import tqdm
import json
from pathlib import Path
from tqdm.auto import tqdm

In [2]:
def read_hyperliquid_l2book_lz4(symbol_file: Path, symbol: str) -> pl.DataFrame:
    dataframe = {
        'time': [],
        'timestamp': [],
        'is_bid': [],
        'price': [],
        'csize': [],
        'depth': [],
        'num_orders_at_level': [], 
    }

    with open(symbol_file, 'rb') as file:
        data = lz4.frame.decompress(file.read())

    records = []
    for line in data.strip().split(b'\n'):
        if line:  # Skip empty lines
            record = json.loads(line)
            records.append(record)

    for record in records:
        time = record['time']
        ver_num = record['ver_num']
        channel = record['raw']['channel']
        record_symbol = record['raw']['data']['coin']
        timestamp = record['raw']['data']['time']
        levels = record['raw']['data']['levels']
        assert ver_num == 1
        assert len(levels) == 2
        assert record_symbol == symbol 
        assert channel == 'l2Book'
        for k in range(2):
            csize = 0 
            for j, level in enumerate(levels[k]): # bids 
                dataframe['time'].append(time)
                dataframe['timestamp'].append(timestamp)
                dataframe['is_bid'].append(True if k == 0 else False)
                dataframe['price'].append(level['px'])
                csize += float(level['sz'])
                dataframe['depth'].append(j)
                dataframe['csize'].append(csize)
                dataframe['num_orders_at_level'].append(level['n'])
    return pl.DataFrame(dataframe).select(
        pl.col('time').cast(pl.Datetime),
        pl.col('price').cast(pl.Float64),
        pl.col('csize').cast(pl.Float64),
        pl.col('depth').cast(pl.Int16),
        pl.col('num_orders_at_level').cast(pl.Int16),
        pl.from_epoch(pl.col('timestamp'), time_unit='ms').alias('created_time'),
    )

In [3]:
dataset_type = ms.DatasetType.HyperliquidPerpL2
raw_path = Path(dataset_type.raw_data_path(""))
hive_path = Path(dataset_type.hive_path(""))
print(raw_path, hive_path)

/bigdata/mnemosyne/hyperliquid/raw/futures/market_data /bigdata/mnemosyne/hyperliquid/lossless/l2


In [4]:
# date = Date(2025, 9, 30)
# date_df = []
# for hour in tqdm(range(24)):
#     hour_path = raw_path / date.strftime('%Y%m%d') / str(hour) / 'l2Book'
#     symbol_files = filter(lambda x: x.is_file and x.suffix == '.lz4', hour_path.glob('*'))
#     for symbol_file in symbol_files:
#         date_df.append(read_hyperliquid_l2book_lz4(symbol_file, symbol_file.stem))
# date_df = pl.concat(date_df)
# date_df.drop('created_time').write_parquet('~/Documents/market.parquet', compression='brotli')

In [5]:
marketdata_root = Path('/bigdata/mnemosyne/hyperliquid/raw/futures/market_data')
dates = [
    Datetime.strptime(p.stem, '%Y%m%d').strftime('%Y-%m-%d')
    for p in marketdata_root.glob('*') if p.is_dir()
]
print(len(dates), 'dates')

for date in tqdm(dates):
    ms._mnemosyne_ext.py_read_hyperliquid_l2book_bydate_to(
        "/bigdata/mnemosyne/hyperliquid/raw/futures/market_data", 
        date, f'/data/mnemosyne/cache/date={date}/0.parquet'
    )

898 dates


: 

In [5]:
date = '2023-04-15'
ms._mnemosyne_ext.py_read_hyperliquid_l2book_bydate_to(
        "/bigdata/mnemosyne/hyperliquid/raw/futures/market_data", 
        date, f'/data/mnemosyne/cache/date={date}/0.parquet'
    )
pl.read_parquet(f'/data/mnemosyne/cache/date={date}/0.parquet')

time,price,csize,depth,num_orders_at_level,is_bid,created_time
datetime[μs],f64,f64,i16,i16,bool,datetime[μs]
2023-04-15 00:00:07.695948,96.345,3.39,0,2,true,2023-04-15 00:00:07.400
2023-04-15 00:00:07.695948,96.316,32.35,1,1,true,2023-04-15 00:00:07.400
2023-04-15 00:00:07.695948,96.296,74.36,2,1,true,2023-04-15 00:00:07.400
2023-04-15 00:00:07.695948,96.249,118.72,3,1,true,2023-04-15 00:00:07.400
2023-04-15 00:00:07.695948,96.248,151.03,4,1,true,2023-04-15 00:00:07.400
…,…,…,…,…,…,…
2023-04-15 23:59:59.235864,2.8788,8923.6,5,1,false,2023-04-15 23:59:58.970
2023-04-15 23:59:59.235864,2.8872,10425.2,6,1,false,2023-04-15 23:59:58.970
2023-04-15 23:59:59.235864,2.8886,11981.5,7,1,false,2023-04-15 23:59:58.970
